In [1]:
from pydantic import BaseModel, Field, field_validator
from typing import Union, List, Optional, Dict
import datetime
import uuid
import tiktoken
tokenizer = tiktoken.encoding_for_model("gpt-3.5-turbo")


# Base class for all bd_types
class BDType(BaseModel):
    source: str = Field("babydragon", description="The source of the data.")
    timestamp: Optional[datetime.datetime] = Field(None, description="When the data was collected or created. If not provided, the current time is used.")
    id: uuid.UUID = Field(default_factory=uuid.uuid4, description="Unique identifier of the data.")
    data_name: Optional[str] = Field(None, description="Name of the data.")
    elements_name: Optional[List[str]] = Field(None, description="Names of the elements if the data is a list.")

    @field_validator("timestamp")
    def set_timestamp(cls, v):
        return v or datetime.datetime.now()

    @field_validator("id")
    def set_id(cls, values, **kwargs):
        if "id" not in values:
            values["id"] = uuid.uuid4()
        return values


class NaturalLanguageSingle(BDType):
    text: str = Field(..., description="The natural language text. It should be less than or equal to `max_tokens` in length when tokenized.")
    max_tokens: int = Field(8000, description="The maximum allowed length of the text in tokens. The default value is 8000.")
    
    @field_validator("text")
    def validate_text(cls, v, info):
        try:
            # Tokenize the text and get the token count
            token_count = len(tokenizer.encode(v))
        except Exception as e:
            raise ValueError("Failed to tokenize text.") from e

        # Get max_tokens from info.data, if not available, default to 8000
        max_tokens = info.data.get("max_tokens", 8000)

        if token_count > max_tokens:
            raise ValueError(f"Text is longer than {max_tokens} tokens.")

        return v


class NaturalLanguageList(BDType):
    texts: List[NaturalLanguageSingle] = Field(..., description="A list of `NaturalLanguageSingle` objects. Each object should pass the validation requirements of the `NaturalLanguageSingle` class.")



In [2]:
from pydantic import BaseModel, Field, FieldValidationInfo, field_validator
from typing import List, Optional, Set

class DiscreteDataInt(BDType):
    alphabet: Optional[Set[int]] = Field(None, description="Set of allowed discrete variables. All elements should be integers.")
    value: int = Field(..., description="The discrete data value. It should be an integer.")

    @field_validator('alphabet')
    def check_alphabet(cls, v):
        if not all(isinstance(item, int) for item in v):
            raise ValueError("All elements in 'alphabet' should be integers.")
        return v

    @field_validator('value')
    def check_value(cls, v, info: FieldValidationInfo):
        alphabet = info.data.get('alphabet')
        if alphabet is not None and v not in alphabet:
            raise ValueError("Value must be in the alphabet.")
        return v


class DiscreteDataStr(BDType):
    alphabet: Optional[Set[str]] = Field(None, description="Set of allowed discrete variables. All elements should be strings.")
    value: str = Field(..., description="The discrete data value. It should be a string.")

    @field_validator('alphabet')
    def check_alphabet(cls, v):
        if not all(isinstance(item, str) for item in v):
            raise ValueError("All elements in 'alphabet' should be strings.")
        return v

    @field_validator('value')
    def check_value(cls, v, info: FieldValidationInfo):
        alphabet = info.data.get('alphabet')
        if alphabet is not None and v not in alphabet:
            raise ValueError("Value must be in the alphabet.")
        return v


In [3]:
alphabet = ["a", "b", "c"]
data = DiscreteDataStr(value="c", alphabet=alphabet)
#now with try except for out of alphabet
try:
    data = DiscreteDataStr(value="d", alphabet=alphabet)
except Exception as e:
    print(e)
int_alphabet = [1, 2, 3]
int_data = DiscreteDataInt(value=2, alphabet=int_alphabet)
#now with try except for out of alphabet
try:
    int_data = DiscreteDataInt(value=4, alphabet=int_alphabet)
except Exception as e:
    print(e)

1 validation error for DiscreteDataStr
value
  Value error, Value must be in the alphabet. [type=value_error, input_value='d', input_type=str]
    For further information visit https://errors.pydantic.dev/2.1.2/v/value_error
1 validation error for DiscreteDataInt
value
  Value error, Value must be in the alphabet. [type=value_error, input_value=4, input_type=int]
    For further information visit https://errors.pydantic.dev/2.1.2/v/value_error


In [4]:
from pydantic import BaseModel, Field, FieldValidationInfo, field_validator
from typing import List, Optional, Union, Set

# The DiscreteDataInt and DiscreteDataStr models are defined as before

class DiscreteDataList(BaseModel):
    alphabet: Optional[Set[Union[int, str]]] = Field(None, description="Set of allowed discrete variables. All elements should be of the same type (either integers or strings).")
    value: List[Union[DiscreteDataInt, DiscreteDataStr]] = Field(..., description="The list of discrete data values. All elements should be either DiscreteDataInt or DiscreteDataStr, not a mix.")

    @field_validator('value')
    def check_alphabets(cls, value, info: FieldValidationInfo):
        list_alphabet = info.data.get('alphabet')
        if list_alphabet is not None:
            for item in value:
                item_alphabet = item.alphabet
                if item_alphabet is not None and not set(item_alphabet).issubset(list_alphabet):
                    raise ValueError(f"Item alphabet {item_alphabet} is not a subset of the list alphabet {list_alphabet}.")
        return value


In [5]:
# Usage:
alphabet = ["a", "b", "c"]
alphabet_2 = ["d"]
data_items = [
        DiscreteDataStr(value="a", alphabet=alphabet),
        DiscreteDataStr(value="b", alphabet=alphabet),
    ]
data_list = DiscreteDataList(value=data_items, alphabet=alphabet)
data_items_wrong = [
        DiscreteDataStr(value="a", alphabet=alphabet),
        DiscreteDataStr(value="b", alphabet=alphabet),
        DiscreteDataStr(value="d", alphabet=alphabet_2),
    ]
try:
    
    data_list = DiscreteDataList(value=data_items_wrong, alphabet=alphabet)
except Exception as e:
    print(e)

1 validation error for DiscreteDataList
value
  Value error, Item alphabet {'d'} is not a subset of the list alphabet {'a', 'c', 'b'}. [type=value_error, input_value=[DiscreteDataStr(source='...habet={'d'}, value='d')], input_type=list]
    For further information visit https://errors.pydantic.dev/2.1.2/v/value_error


In [6]:
class MultiDimensionalDiscrete(BDType):
    value: List[Union[DiscreteDataInt, DiscreteDataStr]] = Field(..., description="The multidimensional discrete data value. It should be a list of either DiscreteDataInt or DiscreteDataStr.")
    type_dictionary: Dict[int, str] = Field(default_factory=dict, description="The helper dictionary containing the type of each dimension of the value list.")
    def __init__(self, **data):
        super().__init__(**data)
        self.type_dictionary = {i: item.__class__.__name__ for i, item in enumerate(self.value)}
        
    @field_validator('value')
    def check_value(cls, value):
        if len(value) < 2:
            raise ValueError("For multidimensional discrete data, size of the list should be at least 2. For less than 2, use DiscreteDataInt or DiscreteDataStr.")
        return value

    

In [7]:
# Create some examples
discrete_int_1 = DiscreteDataInt(value=1)
discrete_int_2 = DiscreteDataInt(value=2)
discrete_str_1 = DiscreteDataStr(value="a")

# This should work (two DiscreteDataInt)
multi_dim_discrete_1 = MultiDimensionalDiscrete(value=[discrete_int_1, discrete_int_2])
print(multi_dim_discrete_1)

# This should work too (one DiscreteDataInt and one DiscreteDataStr)
multi_dim_discrete_2 = MultiDimensionalDiscrete(value=[discrete_int_1, discrete_str_1])
print(multi_dim_discrete_2)

# This should raise a ValueError (only one DiscreteDataInt)
try:
    multi_dim_discrete_3 = MultiDimensionalDiscrete(value=[discrete_int_1])
except ValueError as e:
    print(f"Caught an expected error: {e}")


source='babydragon' timestamp=None id=UUID('9b78b111-bee7-407a-8a95-c9ae9ff6d98d') data_name=None elements_name=None value=[DiscreteDataInt(source='babydragon', timestamp=None, id=UUID('d07daa29-e2f0-4f5d-95a3-fc1428190e50'), data_name=None, elements_name=None, alphabet=None, value=1), DiscreteDataInt(source='babydragon', timestamp=None, id=UUID('bef822ba-f658-4bf9-bdd6-eaff82f3887d'), data_name=None, elements_name=None, alphabet=None, value=2)] type_dictionary={0: 'DiscreteDataInt', 1: 'DiscreteDataInt'}
source='babydragon' timestamp=None id=UUID('f223e2ba-c45e-4708-8e22-fc8f0e6ad46b') data_name=None elements_name=None value=[DiscreteDataInt(source='babydragon', timestamp=None, id=UUID('d07daa29-e2f0-4f5d-95a3-fc1428190e50'), data_name=None, elements_name=None, alphabet=None, value=1), DiscreteDataStr(source='babydragon', timestamp=None, id=UUID('27612c45-69a5-429a-9e78-6fd0a6e8a809'), data_name=None, elements_name=None, alphabet=None, value='a')] type_dictionary={0: 'DiscreteDataInt'

In [8]:
from pydantic import BaseModel, Field, field_validator
from typing import List, Optional, Union, Tuple, Any, Set

class MultiDimensionalDiscreteList(BDType):
    values: List[MultiDimensionalDiscrete] = Field(..., description="The list of multidimensional discrete data values. All elements should be instances of MultiDimensionalDiscrete.")
    joint_alphabet: Optional[Set[Tuple[Any, ...]]] = Field(None, description="Set of tuples representing allowed discrete variable combinations. All elements should be tuples of the same length as the number of dimensions in each joint discrete variable.")

    @field_validator('values')
    def check_type_dictionaries(cls, values):
        first_type_dictionary = values[0].type_dictionary
        for value in values[1:]:
            if value.type_dictionary != first_type_dictionary:
                raise ValueError("All elements in 'values' should have the same 'type_dictionary'.")
        return values

    @field_validator('joint_alphabet')
    def check_joint_alphabet(cls, v, info):
        if v is not None and "values" in info.data:
            expected_tuple_length = len(info.data["values"][0].value)
            for item in v:
                if not isinstance(item, tuple) or len(item) != expected_tuple_length:
                    raise ValueError(f"Each element in 'joint_alphabet' should be a tuple of length {expected_tuple_length}.")
                for dim_value, dim_alphabet in zip(item, [value.alphabet for value in info.data["values"][0].value]):
                    if dim_alphabet is not None and dim_value not in dim_alphabet:
                        raise ValueError(f"Value {dim_value} is not in the alphabet for its dimension.")
        return v


In [9]:
discrete_int1 = DiscreteDataInt(alphabet=[0, 1, 2, 3, 4], value=2)
discrete_int2 = DiscreteDataInt(alphabet=[0, 1, 2, 3, 4], value=3)
discrete_str1 = DiscreteDataStr(alphabet=['a', 'b', 'c', 'd'], value='b')


In [10]:
multi_dim_discrete1 = MultiDimensionalDiscrete(value=[discrete_int1, discrete_str1])
print(multi_dim_discrete1.model_dump_json())

multi_dim_discrete2 = MultiDimensionalDiscrete(value=[discrete_int2, discrete_str1])
multi_dim_list = MultiDimensionalDiscreteList(values=[multi_dim_discrete1, multi_dim_discrete2], joint_alphabet=[(2, 'b'), (3, 'b')])
print(multi_dim_list.model_dump_json())


{"source":"babydragon","timestamp":null,"id":"46113d80-b664-43c7-8784-4eac154de569","data_name":null,"elements_name":null,"value":[{"source":"babydragon","timestamp":null,"id":"9d4840a5-28b8-4a2b-9a65-1186225e44df","data_name":null,"elements_name":null,"alphabet":[0,1,2,3,4],"value":2},{"source":"babydragon","timestamp":null,"id":"a979058a-2d2b-4710-9de7-aee584d38ce1","data_name":null,"elements_name":null,"alphabet":["d","a","c","b"],"value":"b"}],"type_dictionary":{"0":"DiscreteDataInt","1":"DiscreteDataStr"}}
{"source":"babydragon","timestamp":null,"id":"4bf17151-445c-4c35-b7df-183136cacd9a","data_name":null,"elements_name":null,"values":[{"source":"babydragon","timestamp":null,"id":"46113d80-b664-43c7-8784-4eac154de569","data_name":null,"elements_name":null,"value":[{"source":"babydragon","timestamp":null,"id":"9d4840a5-28b8-4a2b-9a65-1186225e44df","data_name":null,"elements_name":null,"alphabet":[0,1,2,3,4],"value":2},{"source":"babydragon","timestamp":null,"id":"a979058a-2d2b-4710

In [11]:
try:
    multi_dim_list_error = MultiDimensionalDiscreteList(values=[multi_dim_discrete1, multi_dim_discrete2], joint_alphabet=[(5, 'b'), (3, 'b')])
except ValueError as e:
    print(e)


1 validation error for MultiDimensionalDiscreteList
joint_alphabet
  Value error, Value 5 is not in the alphabet for its dimension. [type=value_error, input_value=[(5, 'b'), (3, 'b')], input_type=list]
    For further information visit https://errors.pydantic.dev/2.1.2/v/value_error


In [12]:
try:
    discrete_int3 = DiscreteDataInt(alphabet=[0, 1, 2, 3, 4], value=4)
    multi_dim_discrete_error = MultiDimensionalDiscrete(value=[discrete_int2, discrete_int3])
    print(multi_dim_discrete_error.type_dictionary)
    multi_dim_list_error = MultiDimensionalDiscreteList(values=[multi_dim_discrete1, multi_dim_discrete_error])
except ValueError as e:
    print(e)


{0: 'DiscreteDataInt', 1: 'DiscreteDataInt'}
1 validation error for MultiDimensionalDiscreteList
values
  Value error, All elements in 'values' should have the same 'type_dictionary'. [type=value_error, input_value=[MultiDimensionalDiscrete... 1: 'DiscreteDataInt'})], input_type=list]
    For further information visit https://errors.pydantic.dev/2.1.2/v/value_error
